In [3]:
## Featurizer featurizes methods that convert json objects of the appropriate type into features

class Featurizer:
    def __init__(self):
        pass
        #self.name = name

    # takes in text pickle and scrapedate, spits out text frequency
    def dailyTextFreq(self, texts, scrapedate):
        
        a = texts

        seconds_intwo_weeks = 1209600;
        
        # calculate unix time for two weeks prior
        two_weeks_prior = scrapedate - seconds_intwo_weeks

        text_date = 0
        saved_index = 0

        # find the index of the oldest message that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            text_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
            if (two_weeks_prior < text_date):
                saved_index = len(a) - (i+1)
                break

        # divide by 14 to get daily freq
        return float(saved_index+1)/14
    

    # takes in call pickle and scrapedate, spits out call frequency
    def dailyCallFreq(self, calls, scrapedate):

        a = calls

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        call_date = 0
        saved_index = 0

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            call_date = int(json.loads(a[len(a)-(i+1)])['date'].encode('ascii','ignore'))
            if (two_weeks_prior < call_date):
                saved_index = len(a) - (i+1)
                break

        return float(saved_index+1)/14
    
    
    # input is tweets pickle, return master vector
    def embeddingToMastersum(self, tweets):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,len(tweets)):
            try:
                masterSum += self.tweetToEmbedding(tweets[i])
            except:
                pass
        
        return masterSum

    
    # input is one single tweet, returns vector embedding of entire tweet.
    # eg: responseobject.json()[0]
    def tweetToEmbedding(self, tweet):

        q = tweet['text'].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    # input is tweets pickle, returns follow count
    def followerCount(self, tweets):

        followerCount = 0

        # get
        try:
            followerCount = json.loads(tweets[0])['user']['followers_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followerCount

    # input is tweets pickle, returns friend count
    def followingCount(self, tweets):

        followingCount = 0

        try:
            followingCount = json.loads(tweets[0])['user']['friends_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followingCount

    # input is tweets pickle, return avg likes per post for the last 2 weeks
    def twitterLikeFreq(self, tweets, scrapedate):

        a = tweets

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        tweet_date = 0
        saved_index = 0

        for i in range(0,len(a)):

            utc = json.loads(a[len(a)-(i+1)])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))

            if (two_weeks_prior < tweet_date):
                saved_index = len(a) - (i+1)
                break

        sum_of_likes = 0

        for i in range(0, saved_index):
            sum_of_likes += json.loads(tweets[i])['favorite_count']


        return sum_of_likes/14

    # input is tweets pickle, return avg retweets per post for the last 2 weeks
    def twitterRetweetFreq(self, tweets, scrapedate):

        a = tweets

        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        tweet_date = 0
        saved_index = 0

        for i in range(0,len(a)):

            utc = json.loads(a[len(a)-(i+1)])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y")))

            if (two_weeks_prior < tweet_date):
                saved_index = len(a) - (i+1)
                break

        sum_of_retweets = 0

        for i in range(0, saved_index):
            sum_of_likes += json.loads(tweets[i])['retweet_count']


        return sum_of_retweets/14
    
    # input is contacts pickle, returns number of contacts
    def numOfContacts(self, contacts):
        return len(contacts)
    
    # input is instagram pickle, return two features: follows count, followed by count  
    def instagramThings(self, instagram):
        
        followsFollowed = np.zeros((2,))
        
        
        try:
            if(type(json.loads(instagram[0])) == str):
                print("DAMN YOU DAMON!!(ig)")
            else: # its a dictionary like it's supposed to be
                followsFollowed[0] = json.loads(instagram[0])['data']['counts']['follows']
                followsFollowed[1] = json.loads(instagram[0])['data']['counts']['followed_by']
        except IndexError:
            pass
        
        return followsFollowed
    
    # takes in instagramMedia pickle scrape date, spits out filter usage frequency for the past 2 weeks
    def instagramFilterFreq(self, instagramMedia, scrapedate):
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break

                    
        negative_count = 0

        for i in range(0,saved_index):
            if(json.loads(a[i])['filter'] == Normal):
                negative_count += 1

        # filter usage freq = posts with filters count / total post count 
        return ((saved_index+1)-negative_count)/(saved_index+1)
    
    # takes in instagramMedia pickle and scrape date, returns a vector that
    # contains a normalized percentage (0-1) for the usage of the filters
    # listed below for the past 2 weeks: 
    # Valencia, X-Pro II, Hefe, Amaro, Rise, Willow, Crema, Inkwell
    
    # output example: [0.5,0,0,0,0,0.5,0,0] 
    # interpretation: user used valencia half the time, Willow the other half
    # of time, for the past 2 weeks of Instagram posts.
    def instagramFilterVector(self, instagramMedia, scrapedate):
        
        filters = {'Valencia':0,'X-Pro II':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}
        filtervec = np.zeros((8,))
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break

                    
        for i in range(0,saved_index):
            filt = json.loads(a[i])['filter']
            if(filt in filters):
                filters[filt] += 1
            else:
                pass
            
        for i in range(0,8):
            filtervec[i] = filters[list(filters)[i]]

        # percentage of how much the filters are used as a normalized vector
        return filtervec/(saved_index+1)
    
    # takes in InstagramMedia, returns comment and like frequency for the 
    # past 2 weeks.
    def instagramLikeComFreq(self, instagramMedia, scrapedate):
        
        counts = np.zeros((2,))
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break
                
        for i in range(0,saved_index):
            counts[0] += json.loads(instagramMedia[i])['likes']['count']
            counts[1] += json.loads(instagramMedia[i])['comments']['count']
        
            
        # because dividing by 0 is a big No-No
        if (not(counts[0] != 0 and counts[1] != 0)):
            return np.zeros((2,))
        else:
            # likes per post for past 2 weeks
            return counts/saved_index


    # takes in instagramMedia pickle and scrapedate, returns IG post 
    # frequency for the past 2 weeks
    def instagramPostFreq(self, instagramMedia, scrapedate):
        
        seconds_intwo_weeks = 1209600;

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0
        
        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass
            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break
                
        return (saved_index+1)/14
        
        
    # takes instagramMedia, returns pixelwise average values for [H,S,V]
    # (Hue, Saturation, Value) for all posts in the past 2 weeks, the 
    # count of faces as a frequency of faces per picture, for the past
    # 2 weeks as well.
    def averageHSVF(self, instagramMedia, scrapedate):


        seconds_intwo_weeks = 1209600;

        ## ms into secs
        scrapedate = int(str(scrapedate)[:-3]) 

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0

        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass

            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break


        

        if(os.path.exists("./ILLSTOPBLINKINGSOON")):
            shutil.rmtree('./ILLSTOPBLINKINGSOON')

        os.mkdir('./ILLSTOPBLINKINGSOON')

        avgs = np.zeros((4,))

        ## to avoid division by 0
        if(saved_index == 0):
            return avgs



        for i in range(0,saved_index):
            url = json.loads(instagramMedia[i])['images']['thumbnail']['url']
            urllib.request.urlretrieve(url, './ILLSTOPBLINKINGSOON/' + str(i) + '.jpg')


        # face_cascade here is a pre trained classifier for frontal faces 
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

            
        avgHue = 0
        avgSatur = 0
        avgVal = 0
        avgFaces = 0

        ## GODS OF PROGRAMMING, FORGIVE ME FOR THIS TRIPLE NEST

        for k in range(0, saved_index):
            ## BGR and not RGB because imread reads in BGR
            img = cv2.imread('./ILLSTOPBLINKINGSOON/' + str(k) + '.jpg')
            hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
            
            grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(grayImage,  scaleFactor=1.1, minNeighbors=5, flags = cv2.CASCADE_SCALE_IMAGE)
            
            avgFaces += len(faces)
            
            for i in range(0,hsv.shape[0]):
                for j in range(0,hsv.shape[1]):
                    avgHue += hsv[i,j,0]
                    avgSatur += hsv[i,j,1]
                    avgVal += hsv[i,j,2]
                    
        

        sums = [avgHue,avgSatur,avgVal]

        ## 22500 = 150x150 = instagram photo thumbnail shape
        avgs = list(map(lambda x: x/(22500*saved_index), sums)).append(avgFaces/saved_index)

        shutil.rmtree('./ILLSTOPBLINKINGSOON')

        return avgs      

    
    #input is texts pickle, return master vector 
    def embeddingToMastersumText(self, texts):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,len(texts)):
            try:
                masterSum += self.textToEmbedding(texts[i])
            except:
                pass
        
        return masterSum

    
    # input is single text, returns vector embedding of entire text.
    def textToEmbedding(self, text):
        
        q = json.loads(text)["body"].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    # returns [q1,q2,q3,q4,q5,q6,q7,q8,q9] and sum of all these scores
    def labelGenerator(self, phq):
        
        labelVector = np.zeros((10,))
        sumOfScores = 0
        
        print(phq)
        
        for i in range(0,9):
            temp = int(json.loads(phq[0])['Q' + str(i)])
            labelVector[i] = temp
            labelVector[9] += temp
            

In [6]:
import unittest
import os

f = Featurizer()

In [38]:
class TestFeaturizer(unittest.TestCase):
    
    if __name__ == '__main__':
        unittest.main(argv=['first-arg-is-ignored'], exit=False)
    
    def test_dailyTextFreq(self):
        self.assertEqual(3,3)
        
    def test_dailyTextFre(self):
        self.assertEqual(4,4)
        
    def add(self,x,y):
        return x + y
        
    def test_errorexample(self):
        self.assertRaises(TypeError,self.add,"3",4)    

...
----------------------------------------------------------------------
Ran 3 tests in 0.009s

OK


256